### **ETL datos de clientes y sus suscripciones a una plataforma digital.**
El objetivo es generar métricas limpias y confiables para análisis financiero, construyendo un dataset agregado con:
- país
- tipo_suscripcion
- ingreso_promedio_anual
- total_clientes

In [0]:
import pandas as pd

# Carga de archivos CSV
df_clientes = pd.read_csv(
    '/Workspace/Users/eliasbasaldua30@gmail.com/etl-comparative-pipelines/data/raw/clientes_proyecto.csv'
)

df_suscripciones = pd.read_csv(
    '/Workspace/Users/eliasbasaldua30@gmail.com/etl-comparative-pipelines/data/raw/suscripciones_proyecto.csv'
)

# Unión de datasets
df = df_clientes.merge(
    df_suscripciones,
    on='cliente_id',
    how='inner'
)

# Visualización inicial
print(df.head())

# Verificación de valores nulos
print('\nConteo de nulos en columnas:')
print(df.isna().sum())

# Mensajes informativos sobre limpieza de datos
print('\nValores nulos en columnas tipo_suscripcion y estado_suscripcion deben ser eliminados.')
print('Imputamos columnas de edad y pago_mensual con el valor medio.')
print('Imputamos columna pais con la palabra: "Desconocido".')

# Eliminación e imputación de valores nulos
df = df.dropna(subset=['tipo_suscripcion', 'estado_suscripcion'])
df['edad'] = df['edad'].fillna(df['edad'].mean())
df['pago_mensual'] = df['pago_mensual'].fillna(df['pago_mensual'].mean())
df['pais'] = df['pais'].fillna('Desconocido')

# Verificación final de valores nulos
print(df.isna().sum())

# Creación de columna de pago anual
df['pago_anual'] = df['pago_mensual'] * 12

# Filtrado de registros
df_an = df[
    (df['edad'] >= 21) &
    (df['pago_mensual'] > 0) &
    (df['estado_suscripcion'] == 'Activa')
]

# Agrupación y agregaciones
df_an = (
    df_an
    .groupby(['pais', 'tipo_suscripcion'])
    .agg(
        ingreso_promedio_anual=('pago_anual', 'mean'),
        total_clientes=('cliente_id', 'count')
    )
    .reset_index()
)

# Filtros adicionales
df_an = df_an[
    (df_an['ingreso_promedio_anual'] > 500) &
    (df_an['total_clientes'] >= 10)
]

# Ordenamiento de resultados
df_an = df_an.sort_values(
    by='ingreso_promedio_anual',
    ascending=False
)

# Visualización de resultados
print(df_an.head(10))

# Exportación a CSV
df_an.to_csv(
    '/Workspace/Users/eliasbasaldua30@gmail.com/etl-comparative-pipelines/data/processed/ingresos_agregados_pandas.csv',
    index=False
)

